In [23]:
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from astroNN.nn.layers import MCDropout

2023-04-03 13:48:39.434508: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 13:48:39.530549: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-03 13:48:39.530567: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-03 13:48:40.170911: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
file = '../data/pantheon.txt'
syscov_file = '../data/pantheon_errors.txt'
# file = 'data/lcparam_full_long.txt'
df = pd.read_csv(file, sep = " ", usecols=['zcmb', 'mb', 'dmb'])
# df = pd.read_csv(file, sep = " ")
dmag = df[['dmb']]
df = df[['zcmb', 'mb']]
N = len(df.values)
syscov = np.loadtxt(syscov_file, skiprows=1).reshape((N, N))
cov = np.copy(syscov)

In [4]:
df.head()

,zcmb,mb
0,0.50349,22.93445
1,0.49520,22.88020
2,0.67820,24.03770
3,0.86720,24.34685
4,0.79920,24.36050


In [5]:
dmag = dmag.values.reshape(N,)
np.shape(dmag)

(1048,)

In [6]:
np.shape(dmag), np.shape(cov)
np.diag_indices_from(cov)

(array([   0,    1,    2, ..., 1045, 1046, 1047]),
 array([   0,    1,    2, ..., 1045, 1046, 1047]))

In [7]:
cov[np.diag_indices_from(cov)] += dmag**2
xdiag = 1/cov.diagonal() 

In [8]:
sq_errors = np.diag(syscov)
print(np.shape(sq_errors), type(sq_errors))

(1048,) <class 'numpy.ndarray'>


In [9]:
df['errors'] = np.sqrt(sq_errors+dmag**2)
df.head()

,zcmb,mb,errors
0,0.50349,22.93445,0.126758
1,0.49520,22.88020,0.118172
2,0.67820,24.03770,0.206020
3,0.86720,24.34685,0.174903
4,0.79920,24.36050,0.175347


In [10]:
randomize = np.random.permutation(N)
data = df.values[randomize]

In [11]:
z = data[:,0]
y = data[:,1:]
# print(randomize)

In [12]:
scalerz = StandardScaler()
# scalerz = MinMaxScaler(feature_range=(-1,1))
# fit scaler on data
scalerz.fit(z.reshape(-1,1))
# apply transform
z = scalerz.transform(z.reshape(-1,1))

In [13]:
split = 0.8
ntrain = int(split * len(z))
indx = [ntrain]
z_train, z_test = np.split(z, indx)
y_train, y_test = np.split(y, indx)

In [14]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',
                                   min_delta=0.0,
                                   patience=50.0,
                                   restore_best_weights=True)]
#                      tf.keras.callbacks.ReduceLROnPlateau(patience=2)]

In [15]:
HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([50, 100, 150, 200]))
HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([50, 100, 150, 200]))
HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([2, 5, 10]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adadelta']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))
HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([4, 8]))

In [16]:
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
# with tf.summary.FileWriter('logs/hparam_tuning', sess.graph):
#     init = tf.initialize_all_variables()
#     sess.run(init)
    hp.hparams_config(
        hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_BATCHSIZE],
        metrics=[hp.Metric('loss', display_name="Loss")])

2023-04-03 13:48:41.271104: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-03 13:48:41.271387: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-03 13:48:41.271479: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-04-03 13:48:41.271556: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-04-03 13:48:41.271632: W tensorflow/c

In [17]:
def model_regression_dropout(input_z, hparams):
    # Defeine Keras model for regression
    efirst = Dense(hparams[HP_NUM_UNITS1], activation='relu', input_shape=(1,))(input_z)
#     model = tf.keras.models.Sequential()
#     model.add(tf.keras.layers.InputLayer(batch_input_shape=((None, 1))))
    x = Dense(hparams[HP_NUM_UNITS2], activation='relu')(efirst)
#     model.add(Dense(units=hparams[HP_NUM_UNITS1], activation='relu'))
    x = MCDropout(0.3)(x)
    x = Dense(hparams[HP_NUM_UNITS3], activation='relu')(x)
    x = MCDropout(0.3)(x)
#     x = Dense(hparams[HP_NUM_UNITS4], activation='relu')(x)
#     x = MCDropout(0.3)(x)
#     ehidden2 = Dense(hparams[HP_NUM_UNITS3], activation='relu')(ehidden1)
    elast = Dense(2, activation='linear')(x)
        
    return elast


In [18]:
epochs = 200
input_z = tf.keras.layers.Input(shape = (1,))

In [19]:
def train_test_model(hparams):
    model = Model(input_z, model_regression_dropout(input_z, hparams))  
#     model = Model(input_z, hzmodel(input_z, hparams))  
                              
    model.compile(
#             optimizer=hparams[HP_OPTIMIZER],
            optimizer='adam',
            loss='mean_squared_error',
            metrics=['mean_squared_error']
          )
    # Run with 1 epoch to speed things up for demo purposes
    # model.fit(x_train, y_train, epochs=1, callbacks=callbacks) 
    # _, loss = model.evaluate(x_test, y_test)
    model_trained = model.fit(z_train, y_train, epochs=epochs, 
                              batch_size=hparams[HP_BATCHSIZE], verbose=0) 
    
    loss = model_trained.history['mean_squared_error'][-1]

    return loss
    

In [20]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
#     with tf.summary.FileWriter(run_dir):
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)

In [21]:
session_num = 0

for batch_size in HP_BATCHSIZE.domain.values:
    for num_units1 in HP_NUM_UNITS1.domain.values:
        for num_units2 in HP_NUM_UNITS2.domain.values:
            for num_units3 in HP_NUM_UNITS3.domain.values:
#                 for num_units4 in HP_NUM_UNITS4.domain.values:
#       for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
#                     for optimizer in HP_OPTIMIZER.domain.values:
                hparams = {
                  HP_NUM_UNITS1: num_units1,
                  HP_NUM_UNITS2: num_units2,
                  HP_NUM_UNITS3: num_units3,
#                       HP_NUM_UNITS4: num_units4,
                #                       HP_DROPOUT: dropout_rate,
    #                       HP_OPTIMIZER: optimizer,
                  HP_BATCHSIZE: batch_size
                }
                run_name = "run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run('logs/hparam_tuning/' + run_name, hparams)
                session_num += 1

print("Total combinations: {}".format(session_num+1))

--- Starting trial: run-0
{'num_units1': 50, 'num_units2': 50, 'num_units3': 50, 'batch_size': 4}
--- Starting trial: run-1
{'num_units1': 50, 'num_units2': 50, 'num_units3': 100, 'batch_size': 4}
--- Starting trial: run-2
{'num_units1': 50, 'num_units2': 50, 'num_units3': 150, 'batch_size': 4}
--- Starting trial: run-3
{'num_units1': 50, 'num_units2': 50, 'num_units3': 200, 'batch_size': 4}
--- Starting trial: run-4
{'num_units1': 50, 'num_units2': 100, 'num_units3': 50, 'batch_size': 4}
--- Starting trial: run-5
{'num_units1': 50, 'num_units2': 100, 'num_units3': 100, 'batch_size': 4}
--- Starting trial: run-6
{'num_units1': 50, 'num_units2': 100, 'num_units3': 150, 'batch_size': 4}
--- Starting trial: run-7
{'num_units1': 50, 'num_units2': 100, 'num_units3': 200, 'batch_size': 4}
--- Starting trial: run-8
{'num_units1': 50, 'num_units2': 150, 'num_units3': 50, 'batch_size': 4}
--- Starting trial: run-9
{'num_units1': 50, 'num_units2': 150, 'num_units3': 100, 'batch_size': 4}
--- Sta

--- Starting trial: run-82
{'num_units1': 100, 'num_units2': 50, 'num_units3': 150, 'batch_size': 8}
--- Starting trial: run-83
{'num_units1': 100, 'num_units2': 50, 'num_units3': 200, 'batch_size': 8}
--- Starting trial: run-84
{'num_units1': 100, 'num_units2': 100, 'num_units3': 50, 'batch_size': 8}
--- Starting trial: run-85
{'num_units1': 100, 'num_units2': 100, 'num_units3': 100, 'batch_size': 8}
--- Starting trial: run-86
{'num_units1': 100, 'num_units2': 100, 'num_units3': 150, 'batch_size': 8}
--- Starting trial: run-87
{'num_units1': 100, 'num_units2': 100, 'num_units3': 200, 'batch_size': 8}
--- Starting trial: run-88
{'num_units1': 100, 'num_units2': 150, 'num_units3': 50, 'batch_size': 8}
--- Starting trial: run-89
{'num_units1': 100, 'num_units2': 150, 'num_units3': 100, 'batch_size': 8}
--- Starting trial: run-90
{'num_units1': 100, 'num_units2': 150, 'num_units3': 150, 'batch_size': 8}
--- Starting trial: run-91
{'num_units1': 100, 'num_units2': 150, 'num_units3': 200, '

In [22]:
!kill 7439
%tensorboard --logdir logs/hparam_tuning

/bin/bash: line 1: kill: (7439) - No such process
